# Minimal database interaction example using PyMongo

## Load credentials and sensitive data

In [ ]:
import json
# Opening JSON file
with open('../credentials/credentials.json', 'r') as openfile:
    # Reading from json file
    credentials = json.load(openfile)

# Configure the connection to the database and set the collections

In [ ]:
import pymongo
import sys
from pymongo import MongoClient
import pandas as pd

In [ ]:
db = MongoClient(credentials['pymongo_url'], 
                    credentials['pymongo_port'], 
                     username=credentials['pymongo_username'],
                     password=credentials['pymongo_password']).get_database(credentials['pymongo_database_name'])

In [ ]:
current_collection = eval(credentials['meter_current_name'])
power_collection = eval(credentials['meter_power_name'])
voltage_collection = eval(credentials['meter_voltage_name'])

# Query to select dates to calculate the circuit results

In [ ]:
import datetime
import ipywidgets as widgets
from ipywidgets import Layout, Box, Dropdown, Label


# available dates in collection:
power_collection_dates=sorted(power_collection.distinct("date"), key=lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
power_collection_meters=sorted(power_collection.distinct("id"))

start_date_selection= widgets.Select(
                                        options= power_collection_dates,
                                        value='2021-10-25',
                                        # rows=10,
                                        description='start date:',
                                        disabled=False
                                    )

display(start_date_selection)

end_date_selection= widgets.Select(
                                        options= power_collection_dates,
                                        value='2021-10-31',
                                        # rows=10,
                                        description='end date:',
                                        disabled=False
                                    )

display(end_date_selection)

## Get data for the selected timeframe

In [ ]:
user_requested_timestamps= pd.date_range(start_date_selection.value, end_date_selection.value, freq='5Min').tolist()
user_query = { "date": { "$gte": start_date_selection.value , "$lte": end_date_selection.value}}
user_query_power_df = pd.DataFrame(power_collection.find(user_query))
user_query_power_df

## Process the retrieved data

In [ ]:
db_columns=list(user_query_power_df.columns.values)
time_format = "%H:%M:%S"
db_times=[]
db_colums_withoutTime=[]
for col in db_columns:
    try:
        db_times.append(datetime.datetime.strptime(col, time_format).time())
        # print(f"{col} is the correct date string format.")
    except ValueError:
        db_colums_withoutTime.append(col)
        # print(f"{i} is the incorrect date string format.")

header=[times.strftime("%H:%M:%S") for times in db_times]
# header.insert(0,'date')

melted_df=pd.melt(user_query_power_df, id_vars=['date', 'id', 'power_type'], value_vars=header, var_name='time', value_name='power')
# user_query_power_df[user_query_power_df['power_type']=='positive_active'][header]
pivoted_df=pd.pivot_table(melted_df, values='power', columns='power_type', index=['date', 'time', 'id'])\
            .reset_index()
pivoted_df.columns.name=None
pivoted_df.insert(loc=0, column='datetime', value=pd.to_datetime(pivoted_df['date'] + ' ' + pivoted_df['time'], format="%Y-%m-%d %H:%M:%S"))
processed_power_df=pivoted_df.dropna()
processed_power_df

## Filter data corresponding to one of the measurement devices

In [ ]:
procesed_power_df_meas=processed_power_df.query('id=="W4"')
procesed_power_df_meas

In [ ]:
powerSeries=procesed_power_df_meas[['datetime', 'positive_active']].set_index('datetime')
powerSeries15min=powerSeries.resample('15T').mean()
powerSeries15min

In [ ]:
import requests
from io import StringIO
from requests_toolbelt.multipart.encoder import MultipartEncoder

# minimum length required for the input time series
lookback_window = 672

def predict(series, timesteps_ahead, device, power_type, prediction_server_url):
    """
    Function that returns the prediction of the chosen model using the chosen time series.

    Parameters
    ----------
    series
        The pandas.DataFrame to be used for prediction. It must follow the format shown above. 
        Also, it must have at least lookback window timesteps, as the models need to see that
        far back into the past in order to make a prediction. In this case, it must be at least 7 days
        long (672 timesteps, as we use a resolution of 15 minutes)
    timesteps_ahead
        The amount of timesteps ahead to be redicted. Prediction starts from the next timestep after the
        last one of series. So, as we will perform day ahead forecasting, we need timesteps_ahead=96
    power_type
        The power type that will be predicted, 4 models have been trained, and their ports are:
            - 3041 for lgbm w4 positive reactive
            - 3040 for lgbm w4 positive active
            - 3045 for lgbm w6 positive active
            - 3042 for lgbm w6 positive reactive
    prediction_server_url
        The url of the server that hosts the prediction service.
    Returns
    -------
    pandas.DataFrame
        The result of the prediction
    """
    port_sel={}
    port_sel['W4','active']=3040
    port_sel['W4','reactive']=3041
    port_sel['W6','active']=3045
    port_sel['W6','reactive']=3042
    port=port_sel[device,power_type]
    
    #print("Input dataframe:", series)

    #print ("Port : ", port)
    
    # check length
    if len(series) < lookback_window:
        print("Please input a larger historical time series")
        return

    series = series.to_json()
    input_example = MultipartEncoder(
    fields={"n": str(timesteps_ahead), #96 timesteps for 15min resolution = day ahead forecast
            "series": series,
           }
    )

    result = requests.post(
        f"http://{prediction_server_url}:{port}/predict", data=input_example, headers={"Content-Type": input_example.content_type}
    ).text

    df = pd.read_json(StringIO(result))
    return df

In [ ]:
prediction_server_url=credentials['prediction_server_url']

In [ ]:
df_w4_a = predict(powerSeries15min, 96, 'W4','active',prediction_server_url)
df_w4_a

In [ ]:
df_w6_a = predict(powerSeries15min, 96, 'W6','active',prediction_server_url)
df_w6_a